# 0. Set the default lakehouse for notebook to run from pipeline

In [ ]:
%%configure
{ 
    "defaultLakehouse": { 
        "name": {
                  "parameterName": "lakehouseName",
                  "defaultValue": "defaultlakehousename"
        }
    }
}

# 1. Initialize Parameters

In [ ]:
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit
import java.util.UUID
import java.text.SimpleDateFormat
import java.time.{LocalDate, LocalDateTime, Period}
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit
import java.util.Calendar
import java.sql.Timestamp
import io.delta.tables._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.{Window, WindowSpec}
import org.apache.spark.sql.functions.{coalesce, lit, sum, col, _}
import org.apache.spark.sql.types.{StructField, _}
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.storage.StorageLevel

val runId  = "00000000-0000-0000-0000-000000000000"
val workspaceId =  spark.conf.get("trident.workspace.id")
val workspaceName =  "LakeHouseTesting"
val lakehouseId = spark.conf.get("trident.lakehouse.id")
val lakehouseName =   "IMAXDefault"
val sitesStagingTableName = "Sites_Staging"
val sitesFinalTableName = "Sites"
val filesStagingTableName = "Files_Staging"
val filesFinalTableName = "Files"
val endTime  = "2024-11-15T00:00:00Z"
spark.conf.set("spark.sql.caseSensitive", true)// Welcome to your new notebook


# 2. Checking Required Final Tables exists or not

In [ ]:
val lakehouse  = mssparkutils.lakehouse.get(lakehouseName)
val lakehouseId  = lakehouse.id
val workspaceName = notebookutils.runtime.context("currentWorkspaceName")

val filesStagingLocation = s"abfss://${workspaceId}@onelake.dfs.fabric.microsoft.com/${lakehouseId}/Tables/${filesStagingTableName}"
val sitesStagingLocation = s"abfss://${workspaceId}@onelake.dfs.fabric.microsoft.com/${lakehouseId}/Tables/${sitesStagingTableName}"
val sitesFinalLocation = s"abfss://${workspaceId}@onelake.dfs.fabric.microsoft.com/${lakehouseId}/Tables/${sitesFinalTableName}"
val filesFinalLocation = s"abfss://${workspaceId}@onelake.dfs.fabric.microsoft.com/${lakehouseId}/Tables/${filesFinalTableName}"

//Need to attach a lake house before this
val tables = spark.catalog.listTables()
val siteTableCount = tables.filter(col("name") === lit(sitesFinalTableName)  and array_contains(col("namespace"), lakehouseName) ).count()
val filesTableCount = tables.filter(col("name") === lit(filesFinalTableName) and array_contains(col("namespace"), lakehouseName)).count()
val siteStagingTableCount = tables.filter(col("name") === lit(sitesStagingTableName)  and array_contains(col("namespace"), lakehouseName) ).count()
val filesStagingTableCount = tables.filter(col("name") === lit(filesStagingTableName) and array_contains(col("namespace"), lakehouseName)).count()


# 3. Getting Snapshot dates from last successful extracts

In [ ]:
import org.apache.spark.sql.functions.{col, _}
import org.apache.spark.sql.types._
import org.apache.spark.sql.{DataFrame, Row, SparkSession}
import org.apache.spark.storage.StorageLevel

val dtCurrentDateFormatt = DateTimeFormatter.ofPattern("yyyy-MM-dd HH:mm:ss.S")
val dtRequiredtDateFormatt = DateTimeFormatter.ofPattern("yyyy-MM-dd'T'HH:mm:ss'Z'")
var siteDataExists: Boolean = false
var filesDataExists: Boolean = false

val siteSnapshotDate = {
    if (siteTableCount ==1) {
        val dfSites = spark.sql(s"SELECT MAX(SnapshotDate) AS SnapshotDate FROM ${lakehouseName}.${sitesFinalTableName} ")
        val rowSites: Row = dfSites.select("SnapshotDate").head(1)(0)
        if (rowSites.get(0) == null) 
            endTime 
        else  
        {
            siteDataExists = true
            println(s"Sites data Exists: ${siteDataExists}")
            LocalDateTime.parse(rowSites.get(0).toString(), dtCurrentDateFormatt).format(dtRequiredtDateFormatt)
        }
    }
    else {
        endTime
    }
}

val filesSnapshotDate = {
    if (filesTableCount ==1) {
        val dffiles = spark.sql(s"SELECT MAX(SnapshotDate) AS SnapshotDate FROM ${lakehouseName}.${filesFinalTableName} ")
        val rowfiles: Row = dffiles.select("SnapshotDate").head(1)(0)
        if (rowfiles.get(0) == null) 
            endTime 
        else {
            filesDataExists = true
            println(s"files data Exists: ${filesDataExists}")
            LocalDateTime.parse(rowfiles.get(0).toString(), dtCurrentDateFormatt).format(dtRequiredtDateFormatt) 
        }  
    }
    else {
        endTime
    }
}



# 4. Generate View Script for Sites

In [ ]:
val sitesView: String = s"""
CREATE OR ALTER VIEW vw${sitesFinalTableName}   
AS
SELECT  *,[StorageQuotaFriendly] =  (case 
                when StorageQuota < 1024 then concat(StorageQuota, ' B')
                when StorageQuota < 1048576 then concat(ceiling(StorageQuota / 1024.0), ' KB')
                when StorageQuota < 1073741824 then concat(ceiling(StorageQuota / 1048576.0), ' MB')
                when StorageQuota < 1099511627776  then concat(ceiling(StorageQuota / 1073741824.0), ' GB')
                when StorageQuota < 1125899906842624  then concat(ceiling(StorageQuota / 1099511627776.0), ' TB')
                else concat(ceiling(StorageQuota / 1125899906842624.0), ' PB')
            end )
        ,[StorageUsedFriendly] =  (case 
                when StorageUsed < 1024 then concat(StorageUsed, ' B')
                when StorageUsed < 1048576 then concat(ceiling(StorageUsed / 1024.0), ' KB')
                when StorageUsed < 1073741824 then concat(ceiling(StorageUsed / 1048576.0), ' MB')
                when StorageUsed < 1099511627776  then concat(ceiling(StorageUsed / 1073741824.0), ' GB')
                when StorageUsed < 1125899906842624  then concat(ceiling(StorageUsed / 1099511627776.0), ' TB')
                else concat(ceiling(StorageUsed / 1125899906842624.0), ' PB')
            end )
        ,PreviousVersionStorage =  [StorageMetrics_TotalSize]-[StorageMetrics_MetadataSize]-[StorageMetrics_TotalFileStreamSize]
        ,CreatedTimeBands =   (case 
                when [CreatedTime] is null then 'Unknown'
                when DATEDIFF(month, [CreatedTime], [SnapshotDate]) <=1 then 'A = Up to 1 month'
                when DATEDIFF(month, [CreatedTime], [SnapshotDate]) <=3 then 'B = 1 month - 3 months'
                when DATEDIFF(month, [CreatedTime], [SnapshotDate]) <=6 then 'C = 3 months - 6 months'
                when DATEDIFF(month, [CreatedTime], [SnapshotDate]) <=12 then 'D = 6 months - 1 year'
                when DATEDIFF(month, [CreatedTime], [SnapshotDate]) <=24 then 'E = 1 year - 2 years'
                else 'F = Over 2 years'
                end)
        ,LastItemModifedTimeBands =   (case 
                when [RootWeb_LastItemModifiedDate] is null then 'Unknown'
                when DATEDIFF(month, [RootWeb_LastItemModifiedDate], [SnapshotDate]) <=1 then 'A = Up to 1 month'
                when DATEDIFF(month, [RootWeb_LastItemModifiedDate], [SnapshotDate]) <=3 then 'B = 1 month - 3 months'
                when DATEDIFF(month, [RootWeb_LastItemModifiedDate], [SnapshotDate]) <=6 then 'C = 3 months - 6 months'
                when DATEDIFF(month, [RootWeb_LastItemModifiedDate], [SnapshotDate]) <=12 then 'D = 6 months - 1 year'
                when DATEDIFF(month, [RootWeb_LastItemModifiedDate], [SnapshotDate]) <=24 then 'E = 1 year - 2 years'
                else 'F = Over 2 years'
                end)
        ,OwnerStatus = (case 
                when [Owner_Email] is null and [Owner_Name] is null  then 'Missing Email and Name'
                when [Owner_Email] is null then 'Missing Email'
                when [Owner_Name] is null  then 'Missing Name'
                else 'Valid'
                end)
        ,SiteOwnersPresent  = (case 
                when ([Owner_Email] is not null or [Owner_Name] is not null) and  ([SecondaryContact_Email] is not null or [SecondaryContact_Name] is not null)  then 'Primary and Secondary Contact'
                when ([Owner_Email] is not null or [Owner_Name] is not null) and  ([SecondaryContact_Email] is null and  [SecondaryContact_Name] is null) then 'Only Primary Contact'
                when ([Owner_Email] is null and [Owner_Name] is null) and ([SecondaryContact_Email] is not null or [SecondaryContact_Name] is not null)  then 'Only Secondary Contact'    
                else 'Nether Primary nor Secondary Contact'
                end)
        ,StorageQuotaRemaining = [StorageQuota] - [StorageMetrics_TotalSize]
        ,TotalStorageSizeBands = (case 
                when StorageMetrics_TotalSize is null then null
                when StorageMetrics_TotalSize < POWER(10, 5) then 'A = Up to 100KB'
                when StorageMetrics_TotalSize < POWER(10, 6) then 'B = 100KB - 1MB'
                when StorageMetrics_TotalSize < POWER(10, 7) then 'C = 1MB - 10MB'
                when StorageMetrics_TotalSize < POWER(10, 8) then 'D = 10MB - 100MB'
                when StorageMetrics_TotalSize < POWER(10, 9) then 'E = 100MB - 1GB'
                when StorageMetrics_TotalSize < POWER(10, 10) then 'F = 1GB - 10GB'
                when StorageMetrics_TotalSize < POWER(10, 11) then 'G = 10GB - 100GB'
                when StorageMetrics_TotalSize < POWER(10, 12) then 'H = 100GB - 1TB'
                when StorageMetrics_TotalSize < POWER(10, 13) then 'I = 1TB - 10TB'
                else 'J = Over 10TB'
            end )            
        ,[StorageQuotaRemainingFormatted] =  (case 
                when ([StorageQuota] - [StorageMetrics_TotalSize]) < 1024 then concat(([StorageQuota] - [StorageMetrics_TotalSize]), ' B')
                when ([StorageQuota] - [StorageMetrics_TotalSize]) < 1048576 then concat(ceiling(([StorageQuota] - [StorageMetrics_TotalSize]) / 1024.0), ' KB')
                when ([StorageQuota] - [StorageMetrics_TotalSize]) < 1073741824 then concat(ceiling(([StorageQuota] - [StorageMetrics_TotalSize]) / 1048576.0), ' MB')
                when ([StorageQuota] - [StorageMetrics_TotalSize]) < 1099511627776  then concat(ceiling(([StorageQuota] - [StorageMetrics_TotalSize]) / 1073741824.0), ' GB')
                when ([StorageQuota] - [StorageMetrics_TotalSize]) < 1125899906842624  then concat(ceiling(([StorageQuota] - [StorageMetrics_TotalSize]) / 1099511627776.0), ' TB')
                else concat(ceiling(([StorageQuota] - [StorageMetrics_TotalSize]) / 1125899906842624.0), ' PB')
            end )                 
  FROM ${sitesFinalTableName}
""".stripMargin.replaceAll("[\n\r]"," ")
println(sitesView)

# 5. Generate View Script for Files

In [ ]:
val filesView: String = s"""
CREATE OR ALTER VIEW vw${filesFinalTableName}         
    AS      
SELECT * FROM ${filesFinalTableName}
""".stripMargin.replaceAll("[\n\r]"," ")
println(filesView)

# 6. Generate View Script for File Aggs

In [ ]:
val fileAggsView: String = s"""
CREATE OR ALTER VIEW vw${filesFinalTableName}_Aggs  
    AS      
SELECT * FROM ${filesFinalTableName}_Aggs 
""".stripMargin.replaceAll("[\n\r]"," ")
println(fileAggsView)

# 7. Truncate the Staging tables from previous runs if data already exists

In [ ]:
if (siteStagingTableCount ==1) {
    spark.sql(s"DELETE FROM ${lakehouseName}.${sitesStagingTableName} ")
    println(s"Staging table deleted: ${lakehouseName}.${sitesStagingTableName}")
}else {
    println(s"Staging table ${lakehouseName}.${sitesFinalTableName} not found")
}


if (filesStagingTableCount ==1) {
    spark.sql(s"DELETE FROM ${lakehouseName}.${filesStagingTableName} ")
    println(s"Staging table deleted: ${lakehouseName}.${filesStagingTableName}")
}else {
    println(s"Staging table ${lakehouseName}.${filesStagingTableName} not found")
}

# 8. Return snapshot dates back to Pipeline

In [ ]:
import mssparkutils.notebook
val returnData= s"""{\"LakehouseId\": \"${lakehouseId}\", \"SitesStagingTableName\": \"${sitesStagingTableName}\", \"SitesFinalTableName\": \"${sitesFinalTableName}\",  \"SitesSnapshotDate\": \"${siteSnapshotDate}\", \"SitesDataExists\": ${siteDataExists}, \"SitesView\": \"${sitesView}\",  \"FilesStagingTableName\": \"${filesStagingTableName}\", \"FilesFinalTableName\": \"${filesFinalTableName}\", \"FilesSnapshotDate\": \"${filesSnapshotDate}\", \"EndSnapshotDate\": \"${endTime}\", \"FilesDataExists\": ${filesDataExists}, \"FilesView\": \"${filesView}\", \"FileAggsView\": \"${fileAggsView}\"}"""
println(returnData)
mssparkutils.notebook.exit(returnData)